In [2]:
import pandas as pd
import numpy as np
import pickle, os
from dateutil import parser


In [7]:
mimiciv_dir = '../mimic_iv/'
cohort_dir = '../phase1_teamB/'

In [8]:
predischg_interval_hrs = 48
delta_interval_hrs = 24
rare_med_threshold = 10

## Load Data

In [9]:
cohort = pd.read_csv(f'{cohort_dir}final_cohort_with_outcome_labels.csv', low_memory=False)
cohort_ids = set(cohort.hadm_id.unique())

In [10]:
all_admissions = pd.read_csv(f'{mimiciv_dir}core/admissions.csv', low_memory=False)
selected_admissions = all_admissions[all_admissions.hadm_id.isin(cohort_ids)]

In [11]:
d_labitems = pd.read_csv(f'{mimiciv_dir}hosp/d_labitems.csv', low_memory=False)

In [15]:
# For vitals
iter_chartevents = pd.read_csv(f'{mimiciv_dir}icu/chartevents.csv', low_memory=False, iterator=True, chunksize=100000)
d_items = pd.read_csv(f'{mimiciv_dir}icu/d_items.csv', low_memory=False)

In [13]:
iter_emar = pd.read_csv(f'{mimiciv_dir}hosp/emar.csv', iterator=True, chunksize=100000)
selected_emars = pd.concat([chunk[chunk.hadm_id.isin(cohort_ids)] for chunk in iter_emar])

In [14]:
iter_labs = pd.read_csv(f'{mimiciv_dir}hosp/labevents.csv', iterator=True, chunksize=100000)
selected_labevs = pd.concat([chunk[chunk.hadm_id.isin(cohort_ids)] for chunk in iter_labs])

/Users/jmcculloch/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3338: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/Users/jmcculloch/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3338: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/Users/jmcculloch/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3338: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/Users/jmcculloch/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3338: DtypeWarning: Columns (9,12,13) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, resu

In [16]:
selected_labevs.head()

,labevent_id,subject_id,hadm_id,specimen_id,itemid,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments
6257,96618,10007677,27180483.0,83066528,50947,2120-12-08 08:06:00,2120-12-08 10:15:00,1,1.0,NaN,NaN,NaN,NaN,ROUTINE,NaN
7410,96701,10007677,27180483.0,19111229,51678,2120-12-10 06:55:00,2120-12-10 09:33:00,5,5.0,NaN,NaN,NaN,NaN,ROUTINE,NaN
7546,96767,10007677,27180483.0,19319201,50934,2120-12-12 08:14:00,2120-12-12 09:32:00,2,2.0,NaN,NaN,NaN,NaN,STAT,NaN
7973,95893,10007677,27180483.0,72730729,51678,2120-11-24 07:22:00,2120-11-24 10:02:00,8,8.0,NaN,NaN,NaN,NaN,ROUTINE,NaN
7980,96543,10007677,27180483.0,23515865,50947,2120-12-06 16:00:00,2120-12-06 17:09:00,1,1.0,NaN,NaN,NaN,NaN,ROUTINE,NaN


## Admission and Discharge

In [17]:
admittimes = selected_admissions['admittime'].map(parser.parse)
dischtimes = selected_admissions['dischtime'].map(parser.parse)
stay_duration = (dischtimes - admittimes) / np.timedelta64(1, 'h')


In [18]:
# intialise data of lists.
features_data = {'hadm_id': selected_admissions.hadm_id.to_numpy(),
        'stay_duration_hrs': stay_duration.to_numpy(),
        'admittime': admittimes,
         'dischtime': dischtimes};
  
# Create DataFrame
features = pd.DataFrame(features_data)
features = features.set_index('hadm_id')

In [19]:
features.head()

,stay_duration_hrs,admittime,dischtime
hadm_id,,,
26732088,24.433333,2131-09-04 14:26:00,2131-09-05 14:52:00
25141335,19.133333,2186-10-18 17:52:00,2186-10-19 13:00:00
23337747,25.166667,2134-08-11 16:06:00,2134-08-12 17:16:00
26726555,42.083333,2135-05-26 19:25:00,2135-05-28 13:30:00
25401613,50.700000,2123-10-18 07:15:00,2123-10-20 09:57:00


## Medications

In [20]:
med_list = selected_emars.medication.value_counts() # TODO switch to using training set only

In [21]:
first_rare = np.argmax(med_list.to_numpy() < rare_med_threshold)
meds = [k for k in med_list.keys()] # Get list of meds
for i in range(first_rare): # For nonrare meds, get which hadms have the med
    hadms = set(selected_emars.hadm_id[selected_emars.medication == meds[i]].unique())
    features[meds[i]] = features.index.isin(hadms)

rare_meds = set([meds[i] for i in range(first_rare, len(meds))])
hadms = set(selected_emars.hadm_id[selected_emars.medication.isin(rare_meds)].unique())
features["rare_medication"] = features.index.isin(hadms)

In [22]:
features.head()

,stay_duration_hrs,admittime,dischtime,Sodium Chloride 0.9% Flush,Insulin,Heparin,Furosemide,Metoprolol Tartrate,Acetaminophen,Docusate Sodium,...,Isoproterenol,Hydrocortisone study med,Ibuprofen Suspension,modafinil,Cosopt (PF),Stalevo 100,RISperidone (Disintegrating Tablet),Benzocaine 20% Spray,letermovir,rare_medication
hadm_id,,,,,,,,,,,,,,,,,,,,,
26732088,24.433333,2131-09-04 14:26:00,2131-09-05 14:52:00,True,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
25141335,19.133333,2186-10-18 17:52:00,2186-10-19 13:00:00,True,True,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
23337747,25.166667,2134-08-11 16:06:00,2134-08-12 17:16:00,True,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
26726555,42.083333,2135-05-26 19:25:00,2135-05-28 13:30:00,True,False,False,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
25401613,50.700000,2123-10-18 07:15:00,2123-10-20 09:57:00,True,True,False,True,False,True,False,...,False,False,False,False,False,False,False,False,False,False


## Labs

In [33]:
d_labitems[d_labitems.label.str.contains("arterial", case=False, na=False)]


,itemid,label,fluid,category,loinc_code
96,50801,Alveolar-arterial Gradient,Blood,Blood Gas,NaN


In [34]:
lab_itemids = [50862, 51006, 50893, 50882, 50902, 50912, 50931, 50971, 50983, 50885, 50976, 50861, 50863, 50878,#CMP
                51265, 51279, 51301, #CBC 
                50954, 51274, #LFTs
                51237, #INR
                51100, 51078, 51097,#Urine Studies
                50910, 50956, 51288, 50889, #Injury/Inflammatory markers
                50963, # BNP
                50817#, # Oxygen
               # 220227 # Arterial O2 Saturation
              ]
lab_itemnames = d_labitems[d_labitems.itemid.isin(lab_itemids)].label.to_numpy()
for itemid in lab_itemids: 
    if (d_labitems.itemid == itemid).sum() == 0:
        print(itemid)
print(len(lab_itemnames))
print(len(lab_itemids))

29
29


In [36]:
(selected_labevs.itemid == 22022).sum()

0

In [37]:
selected_labevs['charttime'] = pd.to_datetime(selected_labevs['charttime'])

In [38]:
# Example args: features, d_labitems, lab_itemids, selected_labevs
def fetch_by_itemid(features, d_item_table, itemid_list, selected_patients):
    # intialise data of lists.
    # features_data = {'hadm_id': selected_patients.hadm_id.to_numpy()}
    
    # # Create DataFrame
    # features = pd.DataFrame(features_data)
    # features = features.set_index('hadm_id')

    for itemid in itemid_list:
        # Get lab name
        item_name = d_item_table[d_item_table.itemid == itemid].label.to_numpy()[0]
        features.reset_index()
        # Get first and last lab in stay 
        labs = selected_patients.loc[(selected_patients.itemid==itemid) & ~selected_patients.valuenum.isnull()][['hadm_id','charttime','valuenum']]
        labs = labs.merge(features, how='left', left_on= 'hadm_id', right_on= 'hadm_id')
        hrs_before_disch = (labs.dischtime - labs.charttime)/ np.timedelta64(1, 'h')
        labs = labs[(hrs_before_disch > 0) & (hrs_before_disch < labs.stay_duration_hrs)]
        latest_labs = labs.loc[labs.groupby('hadm_id').charttime.idxmax()].set_index('hadm_id')
        initial_labs = labs.loc[labs.groupby('hadm_id').charttime.idxmin()].set_index('hadm_id')
        features = features.join(latest_labs[['charttime', 'valuenum']].add_suffix("_latest"), how='outer')
        features = features.join(initial_labs[['charttime', 'valuenum']].add_suffix("_init"), how='outer')
        features[str(item_name) + " Latest"] = features.valuenum_latest
        delta_valid =  (features.charttime_latest - features.charttime_init) / np.timedelta64(1, 'h') > delta_interval_hrs
        features[str(item_name) + " Delta"] = (features.valuenum_latest - features.valuenum_init)[delta_valid]
        features = features.drop(['charttime_latest', 'charttime_init', 'valuenum_latest', 'valuenum_init'], axis=1)
    return features
features = fetch_by_itemid(features, d_labitems, lab_itemids, selected_labevs)

In [39]:
features.head()

,stay_duration_hrs,admittime,dischtime,Sodium Chloride 0.9% Flush,Insulin,Heparin,Furosemide,Metoprolol Tartrate,Acetaminophen,Docusate Sodium,...,Lipase Latest,Lipase Delta,Sedimentation Rate Latest,Sedimentation Rate Delta,C-Reactive Protein Latest,C-Reactive Protein Delta,NTproBNP Latest,NTproBNP Delta,Oxygen Saturation Latest,Oxygen Saturation Delta
hadm_id,,,,,,,,,,,,,,,,,,,,,
20001395,209.966667,2180-04-23 22:52:00,2180-05-02 16:50:00,True,False,True,True,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,261.0,NaN,NaN,NaN
20002497,262.150000,2121-08-19 19:33:00,2121-08-30 17:42:00,True,True,True,True,True,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,163.0,-1431.0,NaN,NaN
20003491,252.650000,2197-12-18 04:50:00,2197-12-28 17:29:00,True,True,True,True,False,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,17306.0,11164.0,NaN,NaN
20003739,220.716667,2183-10-06 12:41:00,2183-10-15 17:24:00,True,False,True,True,True,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20003880,213.050000,2187-09-16 21:52:00,2187-09-25 18:55:00,True,True,True,True,True,True,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Vitals

In [40]:
# Get events, vitals, and then only events that include those vitals
selected_chartevents = pd.concat([chunk[chunk.hadm_id.isin(cohort_ids)] for chunk in iter_chartevents])
print(selected_chartevents.head())

vitals = d_items[d_items.category.str.contains("vital", case=False, na=False)] # Includes BPM, BP, Temp, SpO2
vital_itemids = list(vitals.itemid)
print(vitals.label)


selected_chartevents_vitals = selected_chartevents[selected_chartevents.itemid.isin(vital_itemids)]
selected_chartevents_vitals['charttime'] = pd.to_datetime(selected_chartevents_vitals['charttime'])
print(selected_chartevents_vitals.head())

      subject_id   hadm_id   stay_id            charttime  \
6881    10007677  27180483  34992648  2120-11-16 23:59:00   
6882    10007677  27180483  34992648  2120-11-17 00:00:00   
6883    10007677  27180483  34992648  2120-11-17 00:00:00   
6884    10007677  27180483  34992648  2120-11-17 00:00:00   
6885    10007677  27180483  34992648  2120-11-17 00:00:00   

                storetime  itemid value  valuenum valueuom  warning  
6881  2120-11-17 00:00:00  223761  98.5      98.5       °F        0  
6882  2120-11-17 02:35:00  220046   130     130.0      bpm        0  
6883  2120-11-17 02:35:00  220047    50      50.0      bpm        0  
6884  2120-11-17 02:35:00  223751   150     150.0     mmHg        0  
6885  2120-11-17 02:35:00  223752    90      90.0     mmHg        0  
1                                  Heart Rate
4                                Heart Rhythm
5            Arterial Blood Pressure systolic
6           Arterial Blood Pressure diastolic
7                Arterial Blo

<ipython-input-40-02dafe133d9e>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_chartevents_vitals['charttime'] = pd.to_datetime(selected_chartevents_vitals['charttime'])


In [41]:
features = fetch_by_itemid(features, d_items, vital_itemids, selected_chartevents_vitals)

In [42]:
features[features["Forehead SpO2 Sensor in Place Latest"] > 0]
# Make sure not everything is NaNs

,stay_duration_hrs,admittime,dischtime,Sodium Chloride 0.9% Flush,Insulin,Heparin,Furosemide,Metoprolol Tartrate,Acetaminophen,Docusate Sodium,...,PAR-Respiration Latest,PAR-Respiration Delta,EtCO2 Latest,EtCO2 Delta,EtCO2 Clinical indication Latest,EtCO2 Clinical indication Delta,Forehead SpO2 Sensor in Place Latest,Forehead SpO2 Sensor in Place Delta,Forehead Sensor Position Changed Latest,Forehead Sensor Position Changed Delta
hadm_id,,,,,,,,,,,,,,,,,,,,,
20040003,386.866667,2181-03-02 17:48:00,2181-03-18 20:40:00,True,False,False,True,True,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN
23680853,272.366667,2133-03-21 04:23:00,2133-04-01 12:45:00,True,False,True,True,False,True,False,...,NaN,NaN,44.0,-3.0,NaN,NaN,1.0,NaN,1.0,NaN
25382044,382.083333,2131-11-13 19:19:00,2131-11-29 17:24:00,True,True,True,True,True,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
26690757,241.916667,2143-01-13 14:55:00,2143-01-23 16:50:00,True,True,True,True,True,False,False,...,NaN,NaN,20.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN
28259228,122.966667,2136-09-28 18:22:00,2136-10-03 21:20:00,True,True,True,True,True,False,True,...,2.0,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,0.0


## Save features to file

In [ ]:
features.join(cohort.setIndex("hadm_id")["admission_num"])

In [43]:
features.to_csv('features.csv')

In [46]:
#cohort.set_index("hadm_id")["admission_num"].head()

KeyError: 'admission_num'